In [66]:
import os
import pandas as pd
from json import loads,dumps
from collections import Counter
import re
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Checking for common ids 

In [67]:
p1_path = "/Users/nida/Documents/GitHub/youtube-kids-ads-25/ad classification/base experiments/"
p2_path = "/Users/nida/Documents/GitHub/youtube-kids-ads-25/ad classification/P2 Cross Model/"

zeroshot_df = pd.read_csv(p1_path + 'pipeline1_base_haleema_propmpt_zeroshot14.csv')
dynamic_fewshot_df = pd.read_csv(p1_path + 'pipeline1_base_dynamicFewShotWithHaleemaPromptAnd2AppAnd2InappExamplesMerged_2299.csv')


davsp_gemini_df = pd.read_csv(p2_path+'Dynamic-Audio-Visual-Sampling_Gemini-1.5_Prompt-C.csv')
davsp_gpt_df = pd.read_csv(p2_path+'Dynamic-Audio-Visual-Sampling_GPT-4o_Prompt-C.csv')
davsp_claude_df = pd.read_csv(p2_path+'Dynamic-Audio-Visual-Sampling_Claude-3.5-Flash_Prompt-C.csv')




In [68]:
all_dfs = {
    "zeroshot":zeroshot_df,
    "dynamic fewshot":dynamic_fewshot_df,
    "davsp gemini": davsp_gemini_df,
    "davsp gpt": davsp_gpt_df,
    "davsp claude":davsp_claude_df
}

In [69]:
for k,v in all_dfs.items():
    print(k,"shape:",v.shape)

zeroshot shape: (2434, 6)
dynamic fewshot shape: (2299, 7)
davsp gemini shape: (2405, 6)
davsp gpt shape: (2407, 5)
davsp claude shape: (2406, 4)


In [70]:
## check for duplicates in any dfs
for name, df in all_dfs.items():
    df_unique = df.drop_duplicates(subset=['Video Id'])
    print(name,df_unique.shape)

zeroshot (2434, 6)
dynamic fewshot (2299, 7)
davsp gemini (2405, 6)
davsp gpt (2407, 5)
davsp claude (2406, 4)


## finding common ids across BOTH pipelines AND across all models

In [71]:

dataframes = list(all_dfs.values())
common_video_ids = set(dataframes[0]['Video Id'])

for df in dataframes[1:]:
    common_video_ids &= set(df['Video Id'])

In [72]:
len(common_video_ids)

2229

In [87]:
with open('common_ids_across_both_pipelines_and_models_ad_task.json','w') as f:
    f.write(dumps(list(common_video_ids)))

In [73]:
shizza_common = pd.read_csv('/Users/nida/Downloads/ad_prob_common_video_ids.csv')
shizza_common

,Video Id
0,-5Nn9YiCFyA
1,-5n4A0ahSPc
2,-6dleqTJs0o
3,-7mSXKxLcMA
4,-8DkqpUK_DQ
...,...
2392,zpE9jKxjnHs
2393,zpjtz3wCWnQ
2394,zsytu0s2hwE
2395,zuuLAQy-FpM


## Checking status of all videos

In [74]:
ad_details_df = pd.read_csv('ad_details_w_tags.csv')
print("Total ads:",ad_details_df.shape)

Total ads: (2520, 15)


In [75]:
ad_details_df.head()

,id,title,description,channelId,channelTitle,thumbnail,tags,categoryId,madeForKids,duration,duration seconds,Primary Tag,Secondary Tag,Coder Rating,Coder Description
0,v81vT7EID3Q,Nangrow HMO,Nangrow HMO,UCX0Sis-gI_OY-Ogz221GGoQ,GrowiingUp,https://i.ytimg.com/vi/v81vT7EID3Q/default.jpg,NaN,22,FALSE,PT5S,5,child directed,Cooking and Food,NaN,ad for a child's dietary supplement
1,rSickeJuceE,nouvelle technologie #Renault #ETech #fullhybrid,découvrez en juin la nouvelle technologie #Ren...,UCKqGfcDbnIDPWTxZ-k-dg3g,Renault Maroc,https://i.ytimg.com/vi/rSickeJuceE/default.jpg,NaN,2,FALSE,PT6S,6,irrelevant,Autos & Vehicles,NaN,ad for car
2,DwNatxJHBGo,දන්ත සංවේදීතාවය නොසලකා හරින්න එපා කල්ගතවෙද්දි ...,දන්ත සංවේදීතාවය නොසලකා හරින්න එපා කල්ගතවෙද්දි ...,UCH3D6zUxAhanFrpVRgzfXuQ,Sensodyne LK,https://i.ytimg.com/vi/DwNatxJHBGo/default.jpg,NaN,22,FALSE,PT6S,6,irrelevant,Health and Fitness,NaN,ad for a toothpaste by sensodyne
3,cJR25ndPunQ,Ditt viktigaste val,NaN,UC4Ky1pwsI1BVMh7Dk5Vwe2g,Kry,https://i.ytimg.com/vi/cJR25ndPunQ/default.jpg,NaN,28,FALSE,PT6S,6,irrelevant,Health and Wellness,NaN,health care centre
4,iviCwMZYxZE,Fyll på ditt SL-kort i appen,Använder du ett SL-kort när du reser? Ladda på...,UCT-K0hbX-rWawfv1H7164fA,KanalSL,https://i.ytimg.com/vi/iviCwMZYxZE/default.jpg,NaN,19,FALSE,PT6S,6,irrelevant,Travel & Adventure,NaN,transport service ad


In [76]:
train_df = pd.read_csv('train_video_ids.csv')
train_ids = train_df['Video id'].to_list()

In [77]:
len(train_ids)

150

In [78]:
video_ids = ad_details_df['id'].to_list()
video_type = []

for v in video_ids:
    if v in train_ids:
        video_type.append('train')
    else:
        video_type.append('test')

Counter(video_type)       

Counter({'test': 2370, 'train': 150})

In [79]:
ad_details_df['Type'] = video_type

In [80]:
for name,df in all_dfs.items():
    
    status = []
    
    this_df_ids = df['Video Id'].to_list()
    
    for v in video_ids:
        if v in this_df_ids:
            status.append("done")
        else:
            status.append("not done")
    
    ad_details_df[name] = status 
    
        
    

In [81]:
for name in all_dfs.keys():
    print(name)
    print(ad_details_df[name].value_counts())
    print("\n")

zeroshot
done        2434
not done      86
Name: zeroshot, dtype: int64


dynamic fewshot
done        2299
not done     221
Name: dynamic fewshot, dtype: int64


davsp gemini
done        2405
not done     115
Name: davsp gemini, dtype: int64


davsp gpt
done        2407
not done     113
Name: davsp gpt, dtype: int64


davsp claude
done        2406
not done     114
Name: davsp claude, dtype: int64




In [82]:
test_df = ad_details_df[ad_details_df['Type']=="test"]

for name in all_dfs.keys():
    
    print(name)
    
    print(test_df[name].value_counts())
    
    print("\n")

zeroshot
done        2284
not done      86
Name: zeroshot, dtype: int64


dynamic fewshot
done        2299
not done      71
Name: dynamic fewshot, dtype: int64


davsp gemini
done        2260
not done     110
Name: davsp gemini, dtype: int64


davsp gpt
done        2259
not done     111
Name: davsp gpt, dtype: int64


davsp claude
done        2257
not done     113
Name: davsp claude, dtype: int64




In [83]:
# ad_details_df.to_csv('ad_task_video_status.csv')

In [84]:

# path = '/Users/nida/Documents/GitHub/youtube-kids-ads-25/ad classification/base experiments/'
# df1 = pd.read_csv(path + 'dynamicFewShotWithHaleemaPromptAnd2AppAnd2InappExamples-21Jan.csv')
# df2 = pd.read_csv(path+ 'pipeline1_base_dynamicFewShotWithHaleemaPromptAnd2AppAnd2InappExamplesMerged.csv')
# merged_dataframe = pd.concat([df1,df2], ignore_index=True)
# merged_dataframe

In [85]:
# merged_dataframe.to_csv(path + 'pipeline1_base_dynamicFewShotWithHaleemaPromptAnd2AppAnd2InappExamplesMerged_2299.csv')